In [1]:
import math, os, sys
import json
import ast
import plotly.express as px
import pandas as pd
from datetime import datetime
sys.path.append('/home/eddyod/programming/activebrainatlas')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "activebrainatlas.settings")
import django
django.setup()

In [2]:
from neuroglancer.models import UrlModel, LayerData, CenterOfMass, Structure
from brain.models import Animal

In [ ]:
def find_values(id, json_repr):
    results = []
    def _decode_dict(a_dict):
        try:
            results.append(a_dict[id])
        except KeyError:
            pass
        return a_dict

    json.loads(json_repr, object_hook=_decode_dict) # Return value ignored.
    return results

In [3]:
id = 210
urlModel = UrlModel.objects.get(pk=id)
prep_id = urlModel.animal
prep = Animal.objects.get(pk=prep_id)

In [4]:
prep.prep_id

'DK54'

In [ ]:
point_data = find_values('annotations', urlModel.url)

In [6]:
json_txt = json.loads(urlModel.url)
layers = json_txt['layers']

In [ ]:
for layer in layers:
    if 'annotations' in layer:
        annotation = layer['annotations']
        d = [row['point'] for row in annotation]
        print(d)


In [9]:
side = 'S'
for layer in layers:
    if 'annotations' in layer:
        lname = layer['name']
        if 'com' in lname.lower():
            annotation = layer['annotations']
            #print(layer)
            #continue
            for com in annotation:
                x = com['point'][0]
                y = com['point'][1]
                z = com['point'][2]
                description = com['description']
                if description.endswith('_L'):
                    structure = description.replace('_L','')
                    side = 'L'
                elif description.endswith('_R'):
                    structure = description.replace('_R','')
                    side = 'R'
                else:
                    structure = description
                    side = 'S'
                    
                structure = Structure.objects.get(abbreviation=structure)
                print(x,y,z,description, structure.id)
                centerOfMass = CenterOfMass(prep=prep, structure=structure,x=x,y=y,section=z,side=side,active=True,created=datetime.now())
                centerOfMass.save()

{'type': 'annotation', 'source': {'url': 'local://annotations', 'transform': {'outputDimensions': {'x': [3.25e-07, 'm'], 'y': [3.25e-07, 'm'], 'z': [2e-05, 'm']}, 'inputDimensions': {'0': [1e-05, 'm'], '1': [1e-05, 'm'], '2': [2e-05, 'm']}, 'operations': [0, 0, 0, 0, 0, 0, 1, 1, 1, 1e-05, 0.0001, 0.5, 5, 0.01, 0.1]}}, 'tool': 'annotatePoint', 'annotations': [{'point': [1029.779052734375, 340.7862854003906, 192.5], 'type': 'point', 'id': '710a4ed6f193e53faa173f77ace7d283f2a51922', 'description': 'SC'}, {'point': [1030.948486328125, 736.0538940429688, 200.5], 'type': 'point', 'id': '8f15ee9bb13fc09f43a3816d7df01d824c8dca77', 'description': 'Pn_L'}, {'point': [1071.8785400390625, 713.834716796875, 270.5], 'type': 'point', 'id': 'f7cd58ca7ed0d0e787af9fc69f74ea4f73a94fb4', 'description': 'Pn_R'}, {'point': [1091.273681640625, 313.3565368652344, 270.5], 'type': 'point', 'id': 'b773e0a30fa42422da3502b6422367a72f4d14b8'}], 'filterBySegmentation': ['segments'], 'name': 'COM'}


In [ ]:
structure = Structure.objects.get(abbreviation='Pn')
print(structure.id)

In [ ]:
reference_centers = sql_controller.get_centers_dict(prep.prep_id)
atlas_box_size=(2010,1490,  447)
align_atlas(reference_centers, reference_scales=(0.00001, 0.00001, 20))

In [ ]:
#df = px.data.iris()
fig = px.scatter_3d(df, x='X', y='Y', z='Section',
              color='Section', opacity=0.7)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()